In [1]:
import sys 
sys.path.append("../")

In [34]:
import pandas as pd 
import datetime as dt
import plotly.graph_objects as go 
from technical.indicators import rsi 
from technical.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [4]:
df_raw.shape

(59621, 14)

In [5]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape

(59621, 14)

In [7]:
df_an = rsi(df_an)
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
0,2015-06-01 00:00:00+00:00,1195,1.09573,1.09628,1.09346,1.09424,1.09565,1.09620,1.09337,1.09415,1.09581,1.09636,1.09355,1.09433,NaN
1,2015-06-01 01:00:00+00:00,922,1.09424,1.09473,1.09305,1.09420,1.09416,1.09465,1.09297,1.09412,1.09433,1.09481,1.09313,1.09427,NaN
2,2015-06-01 02:00:00+00:00,656,1.09418,1.09540,1.09404,1.09509,1.09409,1.09531,1.09395,1.09501,1.09426,1.09549,1.09412,1.09517,NaN
3,2015-06-01 03:00:00+00:00,443,1.09506,1.09556,1.09481,1.09502,1.09498,1.09549,1.09473,1.09495,1.09513,1.09564,1.09489,1.09509,NaN
4,2015-06-01 04:00:00+00:00,813,1.09498,1.09652,1.09491,1.09598,1.09491,1.09644,1.09484,1.09590,1.09505,1.09659,1.09498,1.09605,NaN


In [8]:
df_an = apply_patterns(df_an)

In [9]:
df_an["EMA_200"] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [10]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [11]:
our_cols= ["time",'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c', "ENGULFING", 'direction', 'EMA_200', "RSI_14"]

In [12]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [13]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2015-06-11 07:00:00+00:00,1.12986,1.12996,1.12504,1.12628,1.12637,1.12618,False,-1,1.122955,37.475134
1,2015-06-11 08:00:00+00:00,1.12625,1.12688,1.12410,1.12667,1.12674,1.12660,False,1,1.122998,39.097003
2,2015-06-11 09:00:00+00:00,1.12666,1.12690,1.12462,1.12653,1.12661,1.12645,False,-1,1.123038,38.708834
3,2015-06-11 10:00:00+00:00,1.12650,1.12841,1.12632,1.12667,1.12674,1.12660,True,1,1.123080,39.357232
4,2015-06-11 11:00:00+00:00,1.12670,1.12688,1.12326,1.12337,1.12345,1.12329,True,-1,1.123083,31.025534


In [14]:
BUY = 1 
SELL = -1 
NONE = 0
RSI_LIMIT = 50.0 

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY 
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE
    

In [15]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [16]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    54181
-1     2692
 1     2549
Name: count, dtype: int64

In [17]:
LOSS_FACTOR = -1.0 
PROFIT_FACTOR = 1.5 

def apply_take_profit(row):
    if row.SIGNAL != NONE: 
        return(row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else: 
        return 0.0 
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else: 
        return 0.0

In [18]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [19]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
37,2015-06-12 20:00:00+00:00,1.12539,1.12704,1.12519,1.12681,1.12739,1.12623,True,1,1.123392,56.166680,1,1.128940,1.12539
68,2015-06-16 03:00:00+00:00,1.12721,1.12799,1.12692,1.12794,1.12802,1.12787,True,1,1.123751,59.218660,1,1.129035,1.12721
71,2015-06-16 06:00:00+00:00,1.12800,1.13188,1.12593,1.13155,1.13164,1.13146,True,1,1.123925,69.635974,1,1.136875,1.12800
92,2015-06-17 03:00:00+00:00,1.12534,1.12672,1.12519,1.12618,1.12625,1.12611,True,1,1.124084,55.554384,1,1.127440,1.12534
100,2015-06-17 11:00:00+00:00,1.12528,1.12697,1.12470,1.12688,1.12696,1.12681,True,1,1.124244,56.011451,1,1.129280,1.12528


In [20]:
df_plot = df_slim.iloc[0:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]
markers = ["mid_c", "TP", "SL"]
marker_colors = ["#0000FF", "#00FF00", "#FF0000"]

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = "markers",
        marker=dict(color=marker_colors[i],size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=["RSI_14"], height=250)

In [ ]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.end_price = row.mid_c 
        self.SIGNAL = row.SIGNAL 
        self.TP = row.TP 
        self.SL = row.SL 
        self.result = 0.0 
        self.end_time = row.time
        self.start_time = row.time 
        self.duration = 0 

    def close_trade(self, row, result, trigger_price):
        self.running = False 
        self.result = result 
        self.end_time = row.time
        self.trigger_price = trigger_price


    def update(self, row):
        self.duration += 1 
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
             
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
             


In [22]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]

    if row.SIGNAL != NONE:
       open_trades.append(Trade(row))


In [23]:
vars(closed_trades[0])
df_result = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_result.result.sum()

np.float64(450.0)

In [24]:
df_result.sort_values(by="start_index", inplace=True)

In [25]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [26]:
df_m5.shape

(713626, 14)

In [27]:
df_m5.time.max()

Timestamp('2024-12-30 23:55:00+0000', tz='tzutc()')

In [28]:
df_raw.time.max()

Timestamp('2024-12-30 23:00:00+0000', tz='tzutc()')

In [29]:
from dateutil import parser 

In [30]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time >= time_min) & (df_raw.time <= time_max)]

In [31]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
486472,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
486473,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
486474,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
486475,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
486476,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
486477,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
486478,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
486479,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
486480,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
486481,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [32]:
df_m5_slim = df_m5[["time", "mid_h", "mid_l"]].copy()

In [33]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2015-06-01 00:00:00+00:00,1.09615,1.09556
1,2015-06-01 00:05:00+00:00,1.09628,1.09584
2,2015-06-01 00:10:00+00:00,1.09600,1.09556
3,2015-06-01 00:15:00+00:00,1.09584,1.09503
4,2015-06-01 00:20:00+00:00,1.09510,1.09376


In [45]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [46]:
df_signals["m5_start"] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [47]:
df_signals["start_index_h1"] = df_signals.index

In [48]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
25,2015-06-12 08:00:00+00:00,1.12200,1.12296,1.11512,1.11677,1.11694,1.11660,True,-1,1.123267,26.742318,-1,1.108925,1.12200,2015-06-12 09:00:00+00:00,25
37,2015-06-12 20:00:00+00:00,1.12539,1.12704,1.12519,1.12681,1.12739,1.12623,True,1,1.123392,56.166680,1,1.128940,1.12539,2015-06-12 21:00:00+00:00,37
52,2015-06-15 11:00:00+00:00,1.12170,1.12260,1.12042,1.12112,1.12120,1.12104,True,-1,1.123154,44.762202,-1,1.120250,1.12170,2015-06-15 12:00:00+00:00,52
68,2015-06-16 03:00:00+00:00,1.12721,1.12799,1.12692,1.12794,1.12802,1.12787,True,1,1.123751,59.218660,1,1.129035,1.12721,2015-06-16 04:00:00+00:00,68
71,2015-06-16 06:00:00+00:00,1.12800,1.13188,1.12593,1.13155,1.13164,1.13146,True,1,1.123925,69.635974,1,1.136875,1.12800,2015-06-16 07:00:00+00:00,71


In [49]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c',
       'ENGULFING', 'direction', 'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL',
       'm5_start', 'start_index_h1'],
      dtype='object')

In [50]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l','ask_c', 'bid_c','ENGULFING','EMA_200', 'RSI_14', "direction"], axis=1, inplace=True)

In [51]:
df_signals.head()

,mid_c,SIGNAL,TP,SL,m5_start,start_index_h1
25,1.11677,-1,1.108925,1.12200,2015-06-12 09:00:00+00:00,25
37,1.12681,1,1.128940,1.12539,2015-06-12 21:00:00+00:00,37
52,1.12112,-1,1.120250,1.12170,2015-06-15 12:00:00+00:00,52
68,1.12794,1,1.129035,1.12721,2015-06-16 04:00:00+00:00,68
71,1.13155,1,1.136875,1.12800,2015-06-16 07:00:00+00:00,71


In [52]:
df_signals.rename(columns={
    "mid_c" : "start_price",
    "m5_start": "time"
}, inplace=True)

In [53]:
df_signals.head()

,start_price,SIGNAL,TP,SL,time,start_index_h1
25,1.11677,-1,1.108925,1.12200,2015-06-12 09:00:00+00:00,25
37,1.12681,1,1.128940,1.12539,2015-06-12 21:00:00+00:00,37
52,1.12112,-1,1.120250,1.12170,2015-06-15 12:00:00+00:00,52
68,1.12794,1,1.129035,1.12721,2015-06-16 04:00:00+00:00,68
71,1.13155,1,1.136875,1.12800,2015-06-16 07:00:00+00:00,71


In [54]:
df_m5_slim.head(2)

,time,mid_h,mid_l
0,2015-06-01 00:00:00+00:00,1.09615,1.09556
1,2015-06-01 00:05:00+00:00,1.09628,1.09584


In [56]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on="time", how="left")

In [59]:
merged[merged.SIGNAL.isna()==False].head()
merged.fillna(0, inplace=True)

In [61]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [62]:
merged.head()

,time,mid_h,mid_l,start_price,SIGNAL,TP,SL,start_index_h1
0,2015-06-01 00:00:00+00:00,1.09615,1.09556,0.0,0,0.0,0.0,0
1,2015-06-01 00:05:00+00:00,1.09628,1.09584,0.0,0,0.0,0.0,0
2,2015-06-01 00:10:00+00:00,1.09600,1.09556,0.0,0,0.0,0.0,0
3,2015-06-01 00:15:00+00:00,1.09584,1.09503,0.0,0,0.0,0.0,0
4,2015-06-01 00:20:00+00:00,1.09510,1.09376,0.0,0,0.0,0.0,0


In [71]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1= row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        #self.end_price = row.mid_c 
        self.SIGNAL = row.SIGNAL 
        self.TP = row.TP 
        self.SL = row.SL 
        self.result = 0.0 
        self.end_time = row.time
        self.start_time = row.time 
        self.duration = 1 

    def close_trade(self, row, result, trigger_price):
        self.running = False 
        self.result = result 
        self.end_time = row.time
        self.trigger_price = trigger_price


    def update(self, row):
        self.duration += 1 
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
             
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
             


In [66]:
open_trades_m5 = []
closed_trades_m5= []

for index, row in merged.iterrows():

    if row.SIGNAL != NONE:
       open_trades_m5.append(TradeM5(row))

    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [68]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [70]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2698,25,1.11677,1.12019,-1,1.108925,1.12200,-1.0,2015-06-12 12:10:00+00:00,2015-06-12 09:00:00+00:00,39
1,False,3022,52,1.12112,1.12050,-1,1.120250,1.12170,-1.0,2015-06-15 12:05:00+00:00,2015-06-15 12:00:00+00:00,2
2,False,3214,68,1.12794,1.12711,1,1.129035,1.12721,-1.0,2015-06-16 05:15:00+00:00,2015-06-16 04:00:00+00:00,16
3,False,3250,71,1.13155,1.12783,1,1.136875,1.12800,-1.0,2015-06-16 07:40:00+00:00,2015-06-16 07:00:00+00:00,9
4,False,3502,92,1.12618,1.12523,1,1.127440,1.12534,-1.0,2015-06-17 05:20:00+00:00,2015-06-17 04:00:00+00:00,17


In [72]:
df_res_m5.result.sum()

np.float64(-141.5)

In [ ]:
# with the spread attach to the trades 
